In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

Load the data

In [ ]:
sample = pd.read_csv('/kaggle/input/spaceship-titanic/sample_submission.csv')
train = pd.read_csv('/kaggle/input/spaceship-titanic/train.csv')
test = pd.read_csv('/kaggle/input/spaceship-titanic/test.csv')

In [ ]:
# lets check the training data
train.head()

In [ ]:
# dataframe information
train.info()

# EDA

In [ ]:
# Exploration to understand the features 

# util for getting info about dataframe
def get_info(df):
    for col in df.columns:
        print(f'{col !r:20}{df[col].dtype !r:20} missing : {df[col].isna().sum() :3} percentage : {(df[col].isna().sum())/len(df) :.2f}')
    
print('TRAINING DATA \n','*'*100)
get_info(train)
print('\nTESTING DATA \n','*'*100)
get_info(test)

In [ ]:
# features for category encoding

cat_types = []


print('length of dataframe',len(train))
print('\nNumber of unique values')
print('Feature',' '*12,'Train',' '*4,'Test')
for col in train.select_dtypes(['O']).columns:
    print(f'{col !r:20} {train[col].nunique() !r:10} {test[col].nunique() !r:10}')

print('\n\n')
for col in train.select_dtypes(['O']).columns:
    if train[col].nunique()<5:
        print(f'Feature : {col} \n{train[col].value_counts()}\n')
        cat_types.append(col)


Numerical data stats : Training

In [ ]:
train.select_dtypes(['float64']).describe().iloc[:3,:]

Numerical data stats : Testing

In [ ]:
test.select_dtypes(['float64']).describe().iloc[:3,:]

# Preprocessing

## Encode the data with correct datatype

Encoding each feature with its correct type helps ensure each feature is treated appropriately by whatever functions we use, and makes it easier for us to apply transformations consistently. 

In [ ]:
# let combine the train and test to apply preprocessing steps
# After preprocessing is done we'll split them
train_ = train.set_index('PassengerId')
test_ = test.set_index('PassengerId')
temp = pd.concat([train_,test_])

The numeric features are already encoded correctly (`float` for continuous, `int` for discrete), but the categoricals we'll need to do ourselves.

In [ ]:
for col in temp.select_dtypes(['O']):
    temp[col] = temp[col].astype('category')
    if 'None' not in temp[col].cat.categories:
        temp[col] = temp[col].cat.add_categories('None')

## Handling the missing data

In [ ]:
for col in temp.select_dtypes(['float64']):
    temp[col] = temp[col].fillna(0.0)
    
for col in temp.select_dtypes(['category']):
    temp[col] = temp[col].fillna('None')

In [ ]:
# split the  temp into train and test
train_ = temp.loc[train_.index,:]
test_ = temp.loc[test_.index,:]

Lets get the data back to the way it was before by making `PassengerId` a column 

In [ ]:
train_ = train_.reset_index('PassengerId')
test_ = test_.reset_index('PassengerId')

In training data -- we've the label column `Transported`, let's fix it

In [ ]:
# fixing train labels 
train_['Transported'] = train_['Transported'].astype('bool')
# there are no labels in test data 
test_.drop(columns=['Transported'], inplace=True)

In [ ]:
# TRAINING DATA -- after processing 
train_.info()

In [ ]:
# TEST DATA -- after processing 
test_.info()

## Features and labels

In [ ]:
def get_features_for_ensemble_models(data, drop_feat=[]):
    df = data.copy()
    df = df.drop(columns=drop_feat)

    for col in df.select_dtypes(['category']):
        df[col],_ = df[col].factorize()
    
    return df

X = get_features_for_ensemble_models(train_, drop_feat=['PassengerId','Name','Transported'])
y = train_['Transported'].astype('int')

## Baseline model

lets create a tree based model

In [ ]:
from sklearn.model_selection import cross_val_score

# CLASSIFICATION LINEAR MODELS
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import SGDClassifier

# CLASSIFICATION TREE BASED MODEL
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import GradientBoostingClassifier
from xgboost import XGBClassifier

In [ ]:
model_list = [
    ('RandomForestClassifier',RandomForestClassifier()),
    ('GradientBoostingClassifier',GradientBoostingClassifier()),
    ('XGBClassifier',XGBClassifier(eval_metric='logloss',use_label_encoder=False)),
    ('LogisticRegression',LogisticRegression(max_iter=1000)),
    ('SVC',SVC()),
    ('KNeighborsClassifier',KNeighborsClassifier()),
    ('SGDClassifier',SGDClassifier())
]



def model_performance(X,y,models=model_list):
    for name, model in models:
        cv = cross_val_score(model,X,y,scoring='accuracy',cv=5)
        print(f'{name !r:30} : {np.mean(cv)}')
    
    

In [ ]:
model_performance(X,y,model_list)

## Model improvement 
Ok, We have our baseline scores, now let's try to improve the classification

**How can we improve the model**
* While handling missing values we use `None` for categorical features and '0' for numerical features , we can try other imputations like using mean,mode,etc.. for numerical features and try using most_frequent values for categorical features
* Use standard scaler to standardize the numerical features
* Hyperparameter tuning 
* Throw away poor predictors 
* Check correlation score for insights 
* Cheek mutual information
* Feature engineering 

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

X_ = pd.concat([X,y],axis=1)
plt.figure(figsize=(10,10))
sns.heatmap(X_.corr(), annot=True)
plt.title('Correlation',fontsize=20)
plt.show()

In [ ]:
import matplotlib.pyplot as plt
# mutual information
from sklearn.feature_selection import mutual_info_classif

discrete = []
for col in X.columns:
    discrete.append(X[col].dtype == int)

mi_score = pd.DataFrame(mutual_info_classif(X,y,discrete_features=discrete), index=X.columns, columns=['Mutual information'])

plt.barh(np.arange(len(discrete)), mi_score['Mutual information'])
plt.yticks(ticks=np.arange(len(discrete)),labels=mi_score.index)
plt.xlabel('Mutual information')
mi_score


In [ ]:
#  It looks like these features ['VIP','Age','Destination'] have some interation 
model_performance(X.drop(columns=['VIP','Age','Destination']), y,model_list)

In [ ]:
sns.catplot(x='CryoSleep', y='Age', hue='Transported',data=train_);

In [ ]:
sns.catplot(x='VIP',y='Age',hue='Transported',data=train_ );

In [ ]:
# droping the neumerical features -- not a good idea
X_ = X.drop(columns=['FoodCourt','Spa','VRDeck','ShoppingMall','RoomService'])
model_performance(X_,y,model_list)

In [ ]:
feats = ['FoodCourt','Spa','VRDeck','ShoppingMall','RoomService','Age']
# Lets trying standardize the continuous numerical features, it helps the linear models
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
X_ = scaler.fit_transform(X.loc[:,feats])
model_performance(X_,y,model_list)

In [ ]:
# standardized numerical feats and categorical features

num_feats = ['FoodCourt','Spa','VRDeck','ShoppingMall','RoomService','Age']
cat_feats = ['HomePlanet','CryoSleep','Destination']
# Lets trying standardize the continuous numerical features, it helps the linear models
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import OneHotEncoder
scaler = StandardScaler()
ohe = OneHotEncoder()
scaled = scaler.fit_transform(X.loc[:,num_feats])
one_hot = ohe.fit_transform(X.loc[:,cat_feats])

X_ = np.column_stack((one_hot.todense(),scaled))


model_performance(np.asarray(X_),y,model_list)

In [ ]:
# Let's check for number of values in Cabin
# We want features that help our model to Generalize not to overfit
X['Cabin'].value_counts()

In [ ]:
# Cabin has so many categories , Lets try classification without Cabin as a feature
# Simplicity can break Complexity
model_performance(X.drop(columns=['Cabin']), y, model_list)

## Model pipeline

In [ ]:
from sklearn.impute import SimpleImputer
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import OneHotEncoder

num_imputer_param = {'strategy':'mean', 'fill_value':0}
cat_imputer_param = {'strategy':'most_frequent', 'fill_value':'None'}


def make_model_pipeline(model, num_imputer_param, cat_imputer_param):
    num_feats = ['FoodCourt','Spa','VRDeck','ShoppingMall','RoomService','Age']
    cat_feats = ['HomePlanet','CryoSleep','Destination']

    num_transformer = Pipeline(steps=[('inpute', SimpleImputer(**num_imputer_param)),
                                      ('scaler', StandardScaler())])

    cat_transformer = Pipeline(steps=[('inpute', SimpleImputer(**cat_imputer_param)),
                                      ('encode', OneHotEncoder(handle_unknown='ignore'))])

    transformer = ColumnTransformer(transformers=[
        ('numerical', num_transformer, num_feats),
        ('categorical', cat_transformer, cat_feats)
    ])


    modeling = Pipeline(steps=[
        ('preprocessing',transformer),
        ('modeling',model)
    ])
    
    return modeling


def evaluate_pipeline(X, y, num_imputer_param, cat_imputer_param, models=model_list):
    for name, model in models:
        pipe = make_model_pipeline(model,num_imputer_param, cat_imputer_param)
        cv = cross_val_score(pipe,X,y,scoring='accuracy',cv=5)
        print(f'{name !r:30} : {np.mean(cv)}')
    


Lets try imputing missing values with mean, median, most_frequent etc...


In [ ]:
num_imputer_param = {'strategy':'median'}
cat_imputer_param = {'strategy':'most_frequent'}
clf = make_model_pipeline(SVC(),num_imputer_param, cat_imputer_param)
cv_scores = cross_val_score(clf, train, y, cv=5, scoring='accuracy')
print(cv_scores,'\nmean score',np.mean(cv_scores))

In [ ]:
num_imputer_param = {'strategy':'mean'}
cat_imputer_param = {'strategy':'most_frequent'}
evaluate_pipeline(train,y,num_imputer_param,cat_imputer_param,model_list)

In [ ]:
num_imputer_param = {'strategy':'median'}
cat_imputer_param = {'strategy':'most_frequent'}
evaluate_pipeline(train,y,num_imputer_param,cat_imputer_param,model_list)

## Hyper parameter tuning 

In [ ]:
models__to_tune= [
          ('RandomForest', \
           RandomForestClassifier(), \
           {'modeling__max_depth':[i for i in range(4,12)]}), \
          
          ('LogisticRegression', \
           LogisticRegression(), \
           {'modeling__C':[i*0.1 for i in range(10,15)]}), \
          
          ('GradientBoosting', \
           GradientBoostingClassifier(), \
           {'modeling__n_estimators':[i for i in range(100,300,50)]}), \
          
          ('SVC', \
           SVC(), \
           {'modeling__C':[i for i in range(1,10)]}), \
          
          ('SGDClassifier',SGDClassifier(), \
           {'modeling__warm_start':[True,False], \
            'modeling__early_stopping':[True,False], \
            'modeling__average':[True,False]}), \
          
         ('XGBClassifier', \
           XGBClassifier(use_label_encoder=False, eval_metric='logloss'), \
           {'modeling__colsample_bytree':[0.7], \
            'modeling__colsample_bylevel':[0.5], \
            'modeling__colsample_bynode':[0.7], \
            'modeling__subsample':[0.6,0.7]}) \
         ]

In [ ]:
from sklearn.model_selection import GridSearchCV

num_imputer_param = {'strategy':'mean'}
cat_imputer_param = {'strategy':'most_frequent'}

for name, model, param_grid in models__to_tune:
    pipe = make_model_pipeline(model, num_imputer_param, cat_imputer_param)
    gs = GridSearchCV(pipe, param_grid)
    gs.fit(X,y)
    print(f'{name :30} {gs.best_score_}')

Lets find the best parameters 

In [ ]:
from sklearn.model_selection import GridSearchCV
num_imputer_param = {'strategy':'mean'}
cat_imputer_param = {'strategy':'most_frequent'}
param_grid = {'modeling__n_estimators':[i for i in range(100,500,50)]}
clf = GradientBoostingClassifier()
pipe = make_model_pipeline(clf, num_imputer_param, cat_imputer_param)
gs = GridSearchCV(pipe, param_grid)
gs.fit(train,y)
print(f'GradientBoostingClassifier: {gs.best_score_}')
print(gs.best_params_)

## Lets submit

In [ ]:
# classifier
clf = GradientBoostingClassifier(n_estimators=200)

# # modeling
num_imputer_param = {'strategy':'mean'}
cat_imputer_param = {'strategy':'most_frequent'}
pipe = make_model_pipeline(clf,num_imputer_param,cat_imputer_param)
# # training 
pipe.fit(train,y)

# # prediction
y_preds = pipe.predict(test)

In [ ]:
submission = pd.DataFrame(data={'PassengerId':test['PassengerId'].values,
                                'Transported':y_preds.astype('bool')})

In [ ]:
# save the submission
submission.to_csv('submission.csv', index=False)